Este notebook é um versão da análise deste artigo
http://trasel.com.br/usando-sql-lite-para-encontrar-os-desertos-de-noticias-no-brasil/
usando Pandas.

Vamos utilizar alguns módulos do Python para essa análise.
Estes vem da biblioteca padrão (ou seja, não necessitam ser instalados):

In [1]:
import ftplib
import os
import sqlite3
import zipfile

Os dois próximos pacotes necessitam ser instalados, mas são muito populares para análise de dados.
Para instalá-los, execute
```bash
$ pip install pandas requests xlrd
```

ou, se você estiver usando anaconda e variantes,

```bash
$ conda install pandas requests xlrd
```

(`xlrd` é um pacote utilizado pelo `pandas` para abrir planilhas do Excel)

In [2]:
import pandas as pd
import requests

## Baixando os dados

Precisamos baixar dois arquivos:
- O banco de dados do Atlas da Notícia: https://github.com/voltdatalab/Atlas-Analytics/blob/master/db/atlas.db
- A planilha de todos os municípios brasileiros fornecida pelo IBGE: ftp://geoftp.ibge.gov.br/organizacao_do_territorio/estrutura_territorial/divisao_territorial/2015/dtb_2015.zip

O banco de dados pode ser baixado usando `requests` (útil para lidar com o que está disponível via protocolo `HTTP(S)`),
e vamos também fazer uma checagem para verificar se já temos o arquivo baixado (e assim evitar baixar ele toda vez que reexecutarmos o notebook):

In [3]:
if not os.path.exists(os.path.join('inputs', 'atlas.db')):
    os.makedirs('inputs', exist_ok=True)
    with requests.get("https://github.com/voltdatalab/Atlas-Analytics/raw/master/db/atlas.db") as r:
        with open('inputs/atlas.db', 'wb') as f:
            f.write(r.content)

O banco de dados está disponível agora para uso em `inputs/atlas.db`.

A planilha do IBGE é um pouco mais complicada,
já que é disponível via protocolo `FTP`,
menos utilizado hoje em dia.
Vamos também aproveitar para extrair a planilha do arquivo zipado.

In [4]:
if not os.path.exists(os.path.join('inputs', 'dtb_2015', 'RELATORIO_DTB_BRASIL_MUNICIPIO.xls')):
    municipio_zip = os.path.join('inputs', 'dtb_2015.zip')
    ftp = ftplib.FTP("geoftp.ibge.gov.br")
    ftp.login()
    with open(municipio_zip, 'wb') as f:
        ftp.cwd("organizacao_do_territorio/estrutura_territorial/divisao_territorial/2015")
        ftp.retrbinary("RETR dtb_2015.zip", f.write)
    ftp.close()

    with zipfile.ZipFile(municipio_zip) as municipios:
        municipios.extract('dtb_2015/RELATORIO_DTB_BRASIL_MUNICIPIO.xls', path='inputs')

A planilha está disponível para uso em `inputs/dtb_2015/RELATORIO_DTB_BRASIL_MUNICIPIO.xls`.

Vamos carregar a planilha usando `pandas`,
que facilita a manipulação de dados tabulares (chamados de `DataFrame`, similar aos disponíveis em `R`).
O parâmetro `index_col` permite que usemos a coluna `Nome_Município` como índice,
e também vamos aproveitar para converter os nomes para letras maiúsculas:

In [5]:
municipios_ibge = pd.read_excel('inputs/dtb_2015/RELATORIO_DTB_BRASIL_MUNICIPIO.xls',
                                index_col="Nome_Município")
municipios_ibge.index = municipios_ibge.index.str.upper()
municipios_ibge

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo
Nome_Município,,,,,,,,
ALTA FLORESTA D'OESTE,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015
ALTO ALEGRE DOS PARECIS,11,Rondônia,2,Leste Rondoniense,6,Cacoal,379,1100379
ALTO PARAÍSO,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,403,1100403
ALVORADA D'OESTE,11,Rondônia,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346
ARIQUEMES,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,23,1100023
BURITIS,11,Rondônia,1,Madeira-Guaporé,1,Porto Velho,452,1100452
CABIXI,11,Rondônia,2,Leste Rondoniense,8,Colorado do Oeste,31,1100031
CACAULÂNDIA,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,601,1100601
CACOAL,11,Rondônia,2,Leste Rondoniense,6,Cacoal,49,1100049


Por padrão o `pandas` evita imprimir todos os dados, mas podemos ver que temos 5570 linhas de dados (uma por município).

O próximo passo é carregar o banco de dados do Atlas.
`pandas` permite carregar dados usando SQL,
e nesse caso vamos usar a funcionalidade para ler de um banco de dados SQLite.

In [6]:
cnx = sqlite3.connect('inputs/atlas.db')
atlas = pd.read_sql_query("SELECT * from atlas", cnx, index_col="cidade")
cnx.close()

Existem outras maneiras para carregar esses dados,
mas essa é a mais simples para `SQLite`. Mais detalhes nos seguintes links:
- https://pandas.pydata.org/pandas-docs/stable/io.html#reading-tables
- https://pandas.pydata.org/pandas-docs/stable/io.html#sqlite-fallback

Finalmente temos nossos dados carregados em dois `DataFrame`: `atlas` e `municipios_ibge`.
Para fazer a operação de `LEFT JOIN` podemos usar o método `join`, que por padrão é um `LEFT JOIN`.

In [7]:
joined = municipios_ibge.join(atlas)

Como eu sabia que o `LEFT JOIN` é o padrão? Podemos consultar a ajuda do `pandas` para verificar (veja o parâmetro `how`):

In [8]:
help(atlas.join)

Help on method join in module pandas.core.frame:

join(other, on=None, how='left', lsuffix='', rsuffix='', sort=False) method of pandas.core.frame.DataFrame instance
    Join columns with other DataFrame either on index or on a key
    column. Efficiently Join multiple DataFrame objects by index at once by
    passing a list.
    
    Parameters
    ----------
    other : DataFrame, Series with name field set, or list of DataFrame
        Index should be similar to one of the columns in this one. If a
        Series is passed, its name attribute must be set, and that will be
        used as the column name in the resulting joined DataFrame
    on : column name, tuple/list of column names, or array-like
        Column(s) in the caller to join on the index in other,
        otherwise joins index-on-index. If multiples
        columns given, the passed DataFrame must have a MultiIndex. Can
        pass an array as the join key if not already contained in the
        calling DataFrame. Lik

Outro método útil de `pandas` é `head`,
para visualizar rapidamente os dados sem imprimir todas as linhas da tabela:

In [9]:
joined.head(5)

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo,id,meio,nome,regiao_metropolitana,uf,estado,regiao,pais,fonte
ABADIA DE GOIÁS,52,Goiás,3,Centro Goiano,10,Goiânia,50,5200050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABADIA DOS DOURADOS,31,Minas Gerais,5,Triângulo Mineiro/Alto Paranaíba,19,Patrocínio,104,3100104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABADIÂNIA,52,Goiás,4,Leste Goiano,12,Entorno de Brasília,100,5200100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABAETETUBA,15,Pará,4,Nordeste Paraense,11,Cametá,107,1500107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABAETÉ,31,Minas Gerais,6,Central Mineira,24,Três Marias,203,3100203,id808,Jornal,NOSSO JORNAL,n/d,MG,Minas Gerais,Sudeste,Brasil,Secom/PR


Mas só de visualizar os dados ainda não é claro se temos cidades que não tem veículos de comunicação.
Uma pista que temos é que vemos campos como `id`, `meio` e `nome` com conteúdo `NaN`,
que significa `Not a Number` mas também indica que ao juntar os dados eles não puderam ser preenchidos porque estamos indisponíveis na fonte original (nesse caso, o Atlas).
No caso do post original era um campo `NULL`,
e por isso havia uma instrução `WHERE atlas.cidade IS NULL;` no fim do comando.

Para gerar uma lista de todos os municípios sem veículos de comunicação podemos achar todos os locais da tabela onde temos o valor `NaN`.



In [10]:
missing = joined.loc[joined.isna().any(1)]['Nome_UF']
missing

ABADIA DE GOIÁS                         Goiás
ABADIA DOS DOURADOS              Minas Gerais
ABADIÂNIA                               Goiás
ABAETETUBA                               Pará
ABAIARA                                 Ceará
ABARÉ                                   Bahia
ABATIÁ                                 Paraná
ABAÍRA                                  Bahia
ABDON BATISTA                  Santa Catarina
ABEL FIGUEIREDO                          Pará
ABREU E LIMA                       Pernambuco
ABREULÂNDIA                         Tocantins
ACAIACA                          Minas Gerais
ACAJUTIBA                               Bahia
ACARAPE                                 Ceará
ACARAÚ                                  Ceará
ACARI                     Rio Grande do Norte
ACARÁ                                    Pará
ACAUÃ                                   Piauí
ACEGUÁ                      Rio Grande do Sul
ACORIZAL                          Mato Grosso
ACRELÂNDIA                        

E nesse ponto temos uma lista com o mesmo tamanho da lista original do post.
Yay!
Podemos salvá-la para um CSV usando o método `.to_csv()`:

In [11]:
missing.to_csv("output.csv")

E eis um link para baixar o CSV gerado: [link](output.csv)

## E se usarmos RIGHT JOIN?

No post original há uma discussão sobre os diferentes tipos de `JOIN` disponíveis em SQL.
No caso anterior foi usado `LEFT JOIN`,
mas o que acontece se usarmos um `RIGHT JOIN`?

In [12]:
joined = municipios_ibge.join(atlas, how='right')

In [13]:
missing = joined.loc[joined.isna().any(1)].index
missing.unique()

Index(['ALVORADA D`OESTE', 'AMAMBAÍ', 'ANTONIO PRADO', 'BIGUAÇÚ',
       'CAMPOS DE JORDÃO', 'ELIAS FAUSANTO', 'ESPIGÃO D`OESTE',
       'ESTANCIA VELHA', 'GETULIO VARGAS', 'HERVAL D`OESTE', 'HERVAL D´OESTE',
       'IJUI', 'LUIS EDUARDO MAGALHÃES', 'MACAPA', 'MOGI-MIRIM',
       'NOVA VENÊCIA', 'NÃO INFORMADO NO SITE DA ANJ', 'PARATI',
       'PASANTOS BONS', 'PIÇARRAS', 'POXORÉO', 'RESTINGA SECA', 'REVALDO',
       'SANTA BARBARA DO SUL', 'SANTA BÁRBARA D`OESTE',
       'SANTANA DO LIVRAMENTO', 'SANTO ANTONIO DA PATRULHA',
       'SANTO ANTONIO DAS MISSÕES', 'SANTO AUGUSANTO', 'SANTO CRISANTO',
       'SERAFINA CORREIA', 'SÃO JERONIMO', 'TEOTÔNIA', 'ÁGUAS CLARAS'],
      dtype='object')

Nesse caso temos uma lista das cidades que estão presentes no Atlas,
mas não estão presentes na planilha do IBGE.
De cara um valor se destaca: `NÃO INFORMADO NO SITE DA ANJ`,
que é bastante autodescritivo.
Mas os outros nomes parecem válidos,
então o que está acontecendo?

Muitos tem a ver com crase e acento agudo sendo usados como aspas simples (vide 'HERVAL D`OESTE' e 'HERVAL D´OESTE'):

In [14]:
municipios_ibge[municipios_ibge.index.str.contains("HERVAL D")]

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo
Nome_Município,,,,,,,,
HERVAL D'OESTE,42,Santa Catarina,1,Oeste Catarinense,4,Joaçaba,6702,4206702


Dois casos específicos me chamaram bastante a atenção: `'SANTO AUGUSANTO'` e `'SANTO CRISANTO'`.
Suspeito que deveriam ser `Santo Augusto` e `Santo Cristo`,
duas cidades no interior do RS (e próximas da minha cidade natal).
Vamos primeiro conferir o que o Atlas diz sobre 'SANTO AUGUSANTO':

In [15]:
atlas.loc["SANTO AUGUSANTO"]

id                                 id5310
meio                               Jornal
nome                            O CELEIRO
regiao_metropolitana                  n/d
uf                                     RS
estado                  Rio Grande do Sul
regiao                                Sul
pais                               Brasil
fonte                           Adjori-Rs
Name: SANTO AUGUSANTO, dtype: object

["O Celeiro"](http://www.jornaloceleiro.com.br/) realmente é um jornal da cidade de Santo Augusto - RS.
E a entrada correta no Atlas deveria ser com esse município:

In [16]:
municipios_ibge.loc["SANTO AUGUSTO"]

UF                                               43
Nome_UF                           Rio Grande do Sul
Mesorregião Geográfica                            1
Nome_Mesorregião             Noroeste Rio-grandense
Microrregião Geográfica                           8
Nome_Microrregião                              Ijuí
Município                                     17806
Código Município Completo                   4317806
Name: SANTO AUGUSTO, dtype: object

Do mesmo modo, é o que acontece com `SANTO CRISANTO` e `SANTO CRISTO`:

In [17]:
atlas.loc[atlas.index.str.contains("CRISANTO")]

,id,meio,nome,regiao_metropolitana,uf,estado,regiao,pais,fonte
cidade,,,,,,,,,
SANTO CRISANTO,id5311,Jornal,CORREIO SEMANAL,n/d,RS,Rio Grande do Sul,Sul,Brasil,Adjori-Rs
SANTO CRISANTO,id5312,Jornal,TRIBUNA LIVRE,n/d,RS,Rio Grande do Sul,Sul,Brasil,Adjori-Rs


In [18]:
municipios_ibge.loc["SANTO CRISTO"]

UF                                               43
Nome_UF                           Rio Grande do Sul
Mesorregião Geográfica                            1
Nome_Mesorregião             Noroeste Rio-grandense
Microrregião Geográfica                           1
Nome_Microrregião                        Santa Rosa
Município                                     17905
Código Município Completo                   4317905
Name: SANTO CRISTO, dtype: object

Em ambos os casos a fonte é a AJDORI-RS.
Podemos encontrar qual a fonte das cidades não encontradas na planilha do IBGE:

In [19]:
fontes = atlas.loc[missing.unique()]['fonte']
fontes.unique()

array(['Secom/PR', 'Adjori-Rs', 'Central de Diários', 'Adjori-SP',
       'Adjori-Sc', 'Atlas da Notícia', 'ANJ'], dtype=object)

E também podemos verificar quantos 'erros' vem de cada fonte:

In [20]:
fontes.value_counts()

Secom/PR              33
Adjori-Rs             16
Central de Diários     4
Atlas da Notícia       2
Adjori-SP              1
Adjori-Sc              1
ANJ                    1
Name: fonte, dtype: int64

Idealmente as fontes desses dados deveriam seguir o nome padrão do IBGE,
mas as chances dessa base ter sido gerada manualmente é grande e não é difícil ver como um erro de digitação pode ter ocorrido.

## Corrigindo os outros problemas (coletivamente 😀)

Como isso aqui é um Jupyter Notebook,
você pode continuar verificando os outros problemas.
Dá pra fazer o que eu fiz com os exemplos anteriores,
mas aí vai mais um para o caso de `Santana do Livramento` e um jeito de achar partes do nome da cidade:

In [21]:
atlas.loc[atlas.index.str.contains("LIVRAMENTO")]

,id,meio,nome,regiao_metropolitana,uf,estado,regiao,pais,fonte
cidade,,,,,,,,,
SANTANA DO LIVRAMENTO,id2678,Jornal,A PLATEIA,n/d,RS,Rio Grande do Sul,Sul,Brasil,Secom/PR
SANTANA DO LIVRAMENTO,id5303,Jornal,CORREIO DO PAMPA,n/d,RS,Rio Grande do Sul,Sul,Brasil,Adjori-Rs
SANTANA DO LIVRAMENTO,id5738,Jornal,A PLATÉIA,n/d,RS,Rio Grande do Sul,Sul,Brasil,Central de Diários


In [22]:
municipios_ibge.loc[municipios_ibge.index.str.contains("LIVRAMENTO")]

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo
Nome_Município,,,,,,,,
LIVRAMENTO,25,Paraíba,2,Borborema,10,Cariri Ocidental,8505,2508505
LIVRAMENTO DE NOSSA SENHORA,29,Bahia,6,Centro Sul Baiano,25,Livramento do Brumado,19504,2919504
SANT'ANA DO LIVRAMENTO,43,Rio Grande do Sul,6,Sudoeste Rio-grandense,30,Campanha Central,17103,4317103
NOSSA SENHORA DO LIVRAMENTO,51,Mato Grosso,4,Centro-Sul Mato-grossense,17,Cuiabá,6109,5106109


Eu tentei procurar no site do Atlas sobre como corrigir os erros na base deles,
mas aparentemente os scripts que geram o banco de dados vem de outro lugar que não está no repositório de um jeito reproduzível (só os dados em CSV estão disponíveis) e não pude abrir um pull request no GitHub.
Abri um issue em https://github.com/voltdatalab/Atlas-Analytics/issues
para tentar descobrir qual o melhor jeito de corrigir.